In [1]:
import pandas as pd
import networkx as nx
from tqdm import tqdm
from sklearn.linear_model import LinearRegression

In [2]:
# Чтение CSV-файла и создание графа
n=20000
df = pd.read_csv(f'train_attr_{n}.csv')
G = nx.Graph()
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    node_id = str(row['ego_id']) + '.' + str(row['u'])
    G.add_node(node_id, name=node_id,
               features=[int(row['age']), int(row['city_id']), int(row['sex']), int(row['school']), int(row['university']),
                         int(row['cnt_2']), int(row['cnt_1']), float(row['sex_rel_freq_1']),
                         float(row['sex_rel_freq_2']), int(row['cat_age']),
                         float(row['city_rel_freq']), float(row['school_rel_freq']),
                         float(row['uni_rel_freq'])])

# Создание ребер
df = pd.read_csv(f'train_{n}.csv', dtype={"ego_id":"int64", "u":"int16", "v":"int16", "t":"float", "x1":"float", "x2":"float", "x3":"int16"})
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    p1_id = str(int(row['ego_id'])) + '.' + str(int(row['u']))
    p2_id = str(int(row['ego_id'])) + '.' + str(int(row['v']))
    G.add_edge(p1_id, p2_id, became_when=float(row['t']), x1=float(row['x1']),
               x2=float(row['x2']), x3=float(row['x3']))

100%|████████████████████████████| 39763717/39763717 [17:55<00:00, 36968.17it/s]


In [3]:
df = pd.read_csv('test_attr_full.csv')
G1 = nx.Graph()
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    node_id = str(row['ego_id']) + '.' + str(row['u'])
    G1.add_node(node_id, name=node_id,
                features=[int(row['age']), int(row['city_id']), int(row['sex']), int(row['school']), int(row['university']),
                          int(row['cnt_2']), int(row['cnt_1']), float(row['sex_rel_freq_1']),
                          float(row['sex_rel_freq_2']), int(row['cat_age']),
                          float(row['city_rel_freq']), float(row['school_rel_freq']),
                          float(row['uni_rel_freq'])])
    
# Создание ребер
df = pd.read_csv('test.csv', dtype={"ego_id":"int64", "u":"int16", "v":"int16", "t":"float", "x1":"float", "x2":"float", "x3":"int16"})
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    p1_id = str(int(row['ego_id'])) + '.' + str(int(row['u']))
    p2_id = str(int(row['ego_id'])) + '.' + str(int(row['v']))
    G1.add_edge(p1_id, p2_id, became_when=float(row['t']), x1=float(row['x1']),
                x2=float(row['x2']), x3=float(row['x3']))

100%|████████████████████████████| 40548780/40548780 [34:05<00:00, 19822.83it/s]


In [5]:
import catboost
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import  LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
import implicit

In [ ]:
df = pd.read_csv(f'train_{n}.csv', dtype={"ego_id":"int64", "u":"int16", "v":"int16", "t":"float", "x1":"float", "x2":"float", "x3":"int16"})
X = []
y = []
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    p1_id = str(int(row['ego_id'])) + '.' + str(int(row['u']))
    p2_id = str(int(row['ego_id'])) + '.' + str(int(row['v']))
    if(G.nodes[p1_id] != {} and G.nodes[p2_id] != {}):
        X.append(G.nodes[p1_id]['features'] + G.nodes[p2_id]['features'])
        y.append(G.edges[p1_id, p2_id]['x1'])
    else:
        X.append( [0]*26 )
        y.append(0)

  6%|█▊                           | 2415997/39763717 [01:13<23:36, 26360.87it/s]

In [ ]:
# X = u_factors
# Обучение модели
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
linear = LinearRegression()

In [ ]:
%time
linear.fit(X_train, y_train)

In [ ]:
%time
y_pred = linear.predict(X_test)

In [ ]:
$time
X_train_u, X_test_u, y_train_u, y_test_u = train_test_split(X, y, test_size=0.3, random_state=0)
random_forest = RandomForestRegressor(n_estimators=100, random_state=0)
random_forest.fit(X_train_u, y_train_u)
y_pred = random_forest.predict(X_test_u)

In [ ]:
%time
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X, y, test_size=0.3, random_state=0)
sdg = SGDRegressor(n_estimators=100, random_state=0)
sdg.fit(X_train_s, y_train_s)
y_pred_u = sdg.predict(X_test_s)

In [ ]:
# Оценка качества модели
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('RMSE:', rmse) 

In [12]:
# Оценка качества модели
from sklearn.metrics import mean_squared_error
rmse_u = mean_squared_error(y_test_u, y_pred_u, squared=False)
print('RMSE:', rmse_u) 

RMSE: 0.03631361365135503


In [ ]:
# Оценка качества модели
from sklearn.metrics import mean_squared_error
rmse_s = mean_squared_error(y_test_s, y_pred_s, squared=False)
print('RMSE:', rmse_s) 

In [ ]:
df = pd.read_csv('submission.csv', dtype={"ego_id":"int64", "u":"int16", "v":"int16"})
X = []
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    p1_id = str(int(row['ego_id'])) + '.' + str(int(row['u']))
    p2_id = str(int(row['ego_id'])) + '.' + str(int(row['v']))
    if(G1.nodes[p1_id] != {} and G1.nodes[p2_id] != {}):
        X.append(G1.nodes[p1_id]['features'] + G1.nodes[p2_id]['features'])
    elif(G.nodes[p1_id] != {} and G.nodes[p2_id] != {}):
        X.append(G1.nodes[p1_id]['features'] + G1.nodes[p2_id]['features'])
    else:
        X.append( [0]*26 )
        y.append(0)
y = linear.predict(X)
df['x1'] = y
df.to_csv('submission_for_cup_linear.csv', index=False)

In [ ]:
df = pd.read_csv('submission.csv', dtype={"ego_id":"int64", "u":"int16", "v":"int16"})
X = []
y = []
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    p1_id = str(int(row['ego_id'])) + '.' + str(int(row['u']))
    p2_id = str(int(row['ego_id'])) + '.' + str(int(row['v']))
    if(G1.nodes[p1_id] != {} and G1.nodes[p2_id] != {}):
        X.append(G1.nodes[p1_id]['features'] + G1.nodes[p2_id]['features'])
    elif(G.nodes[p1_id] != {} and G.nodes[p2_id] != {}):
        X.append(G1.nodes[p1_id]['features'] + G1.nodes[p2_id]['features'])
    else:
        X.append( [0]*26 )
        y.append(0)
y = random_forest.predict(X)
df['x1'] = y
df.to_csv('submission_for_cup_forest.csv', index=False)

In [ ]:
df = pd.read_csv('submission.csv', dtype={"ego_id":"int64", "u":"int16", "v":"int16"})
X = []
y = []
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    p1_id = str(int(row['ego_id'])) + '.' + str(int(row['u']))
    p2_id = str(int(row['ego_id'])) + '.' + str(int(row['v']))
    if(G1.nodes[p1_id] != {} and G1.nodes[p2_id] != {}):
        X.append(G1.nodes[p1_id]['features'] + G1.nodes[p2_id]['features'])
    elif(G.nodes[p1_id] != {} and G.nodes[p2_id] != {}):
        X.append(G1.nodes[p1_id]['features'] + G1.nodes[p2_id]['features'])
    else:
        X.append( [0]*26 )
        y.append(0)
y = sdg.predict(X)
df['x1'] = y
df.to_csv('submission_for_cup_sdg.csv', index=False)